In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder ,StandardScaler , LabelEncoder
from sklearn.feature_selection import mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

import os
import math

In [2]:
train = pd.read_csv("../home-data-for-ml-course/train.csv")
test = pd.read_csv("../home-data-for-ml-course/test.csv")
submission = pd.read_csv("../home-data-for-ml-course/sample_submission.csv")
train.columns = train.columns.str.strip()



In [3]:

# data = train.select_dtypes(include=["number"])
# num_cols = len(data.columns)
# cols = 3 
# rows = (num_cols // cols) + (num_cols % cols > 0)  

# fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
# axes = axes.flatten()


# for i, col in enumerate(data.columns):
#     sns.boxplot(x=data[col], ax=axes[i], width=0.3, flierprops={'marker': 'o', 'markerfacecolor': 'red', 'markersize': 5})
#     axes[i].set_title(f"Box Plot de {col}")

# # Supprimer les axes inutilisés
# for j in range(i + 1, len(axes)):
#     fig.delaxes(axes[j])

# plt.tight_layout()
# plt.show()

In [4]:
cols_to_numeric = [
        "LotFrontage", "MasVnrArea", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF",
        "TotalBsmtSF", "BsmtFullBath", "GarageCars", "BsmtHalfBath", "GarageYrBlt"
    ]
for col in cols_to_numeric:
    if col in train.columns:
        train[col] = pd.to_numeric(train[col], errors="coerce")
        test[col] = pd.to_numeric(test[col], errors="coerce")
        
cat_cols = train.select_dtypes(include=['object']).columns
for cat_col in cat_cols:
    train[cat_col] = train[cat_col].fillna("NA")
    test[cat_col] = test[cat_col].fillna("NA")


In [5]:
train=train.drop(train[(train['GrLivArea']>3000) & (train['SalePrice']<300000)].index)
train=train.drop(train[(train['OpenPorchSF']>500) & (train['SalePrice']<100000)].index)
train=train.drop(train[(train['BsmtFinSF1']>5000) & (train['SalePrice']<300000)].index)
train=train.drop(train[(train['YearBuilt']<1900) & (train['SalePrice']>400000)].index)
train=train.drop(train[(train['TotalBsmtSF']>3000) & (train['SalePrice']<300000)].index)
train=train.drop(train[(train['OverallCond']==2) & (train['SalePrice']>300000)].index)
train = train.drop(train[(train['TotRmsAbvGrd'] >13) 
                                  & (train['SalePrice'] < 250000)].index)

In [6]:


test = pd.concat([test, submission["SalePrice"]], axis=1)
data = pd.concat([train , test])

y_SalePrice = data['SalePrice']

In [7]:
data['MSSubClass'] = data['MSSubClass'].astype(str)

data['time'] = data['YrSold'] - data['YearBuilt']

data['Total_Area'] = data['GrLivArea'] + data['TotalBsmtSF']

data['Final_Area'] = data['Total_Area'] + data['GarageArea']

data['Overall_Plus'] = data['OverallQual'] + data['OverallCond']

data['Overall_Mi'] = data['OverallQual'] - data['OverallCond']

data['area'] = data['Final_Area'] + data['LotArea'] * 0.1

data['Lack_of_feature_index'] = data[['Street', "Alley", "MasVnrType", "GarageType", "MiscFeature", 'BsmtQual', 'FireplaceQu','PoolQC','Fence']].isnull().sum(axis=1) + \
                                 (data["MasVnrType"] == 'None') + (data["CentralAir"] == 'No')

data["MiscFeatureExtended"] = (data["PoolQC"].notnull()*1 + data["MiscFeature"].notnull()*1 + data["Fence"].notnull()*1).astype('int64')

data["Has_Alley"] = data["Alley"].notnull().astype('int64')

data['Lot_Occupation'] = data['GrLivArea'] / data['LotArea']

data['Number_Of_Floors'] = (data['TotalBsmtSF'] != 0).astype('int64') + (data['1stFlrSF'] != 0).astype('int64') + (data['2ndFlrSF'] != 0).astype('int64')

data['Total_Close_Live_Area'] = data['GrLivArea'] + data['TotalBsmtSF']

data['Outside_Live_Area'] = data['WoodDeckSF'] + data['OpenPorchSF'] + data['EnclosedPorch'] + data['3SsnPorch'] + data['ScreenPorch']

data['Total_Usable_Area'] = data['Total_Close_Live_Area'] + data['Outside_Live_Area']

data['Area_Quality_Indicator'] = data['Total_Usable_Area'] * data['OverallQual']

data['Area_Quality_Cond_Indicator'] = data['Total_Usable_Area'] * data['OverallQual'] * data['OverallCond']

data['TotalBath'] = (data['FullBath'] + (0.5 * data['HalfBath']) + data['BsmtFullBath'] + (0.5 * data['BsmtHalfBath']))

data['Has_garage'] = data['GarageYrBlt'].notnull().astype('int64')

data['House_Age'] = data['YrSold'] - data['YearBuilt']

data['Is_Remodeled'] = (data['YearBuilt'] != data['YearRemodAdd']).astype('int64')

data['HasBsmt'] = data['BsmtQual'].notnull().astype('int64')

data['Quality_condition'] = data['OverallQual'] * data['OverallCond']

data['Quality_conditition_2'] = data['OverallQual'] + data['OverallCond']

data['House_Age2'] = data['YrSold'] - data['YearRemodAdd']


In [8]:
data = data.drop(columns = ['Id', 'SalePrice'])


In [9]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)  

missing_percentage = data.isnull().mean() * 100
print(missing_percentage)

MSSubClass                      0.000000
MSZoning                        0.000000
LotFrontage                    16.712517
LotArea                         0.000000
Street                          0.000000
Alley                           0.000000
LotShape                        0.000000
LandContour                     0.000000
Utilities                       0.000000
LotConfig                       0.000000
LandSlope                       0.000000
Neighborhood                    0.000000
Condition1                      0.000000
Condition2                      0.000000
BldgType                        0.000000
HouseStyle                      0.000000
OverallQual                     0.000000
OverallCond                     0.000000
YearBuilt                       0.000000
YearRemodAdd                    0.000000
RoofStyle                       0.000000
RoofMatl                        0.000000
Exterior1st                     0.000000
Exterior2nd                     0.000000
MasVnrType      

In [10]:
data['LotFrontage'].fillna(data['LotFrontage'].median(), inplace=True)
data['GarageYrBlt'].fillna(data['GarageYrBlt'].median(), inplace=True)
data['BsmtFullBath'].fillna(data['BsmtFullBath'].mode()[0], inplace=True)
data['GarageType'].fillna(data['GarageType'].mode()[0], inplace=True)
data['GarageCond'].fillna(data['GarageCond'].mode()[0], inplace=True)
data['GarageQual'].fillna(data['GarageQual'].mode()[0], inplace=True)
data['BsmtFinType2'].fillna(data['BsmtFinType2'].mode()[0], inplace=True)
data['BsmtExposure'].fillna(data['BsmtExposure'].mode()[0], inplace=True)
data['BsmtCond'].fillna(data['BsmtCond'].mode()[0], inplace=True)
data['MSZoning'] = data['MSZoning'].fillna(data['MSZoning'].mode())
data['MasVnrArea'].fillna(data['MasVnrArea'].mode()[0], inplace=True)
data['FireplaceQu'].fillna(data['FireplaceQu'].mode()[0], inplace=True)
data['Electrical'].fillna(data['Electrical'].mode()[0], inplace=True)
data['TotalSF'] = data['TotalBsmtSF'] + data['1stFlrSF'] + data['2ndFlrSF']
data['TotalBathrooms'] = data['FullBath'] + (0.5 * data['HalfBath']) + data['BsmtFullBath'] + (0.5 * data['BsmtHalfBath'])



cols_to_fill = ['BsmtHalfBath' ,
    'Functional', 'Utilities', 'BsmtUnfSF', 
    'GarageArea', 'Exterior1st', 'Exterior2nd', 'TotalBsmtSF', 'GarageCars', 
    'Electrical', 'KitchenQual', 'SaleType', 'BsmtFinSF1', 'BsmtFinSF2'
]

# Remplacer les valeurs manquantes par le mode dans les colonnes spécifiées
for col in cols_to_fill:
    if col in data.columns:  # Vérifie si la colonne existe dans le DataFrame
        mode_value = data[col].mode()[0]  # On prend le premier mode trouvé
        data[col].fillna(mode_value, inplace=True)
        
        



/var/folders/rg/r64x99yx2k91vgyysnwnb7pc0000gn/T/ipykernel_26925/1118281666.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['LotFrontage'].fillna(data['LotFrontage'].median(), inplace=True)
/var/folders/rg/r64x99yx2k91vgyysnwnb7pc0000gn/T/ipykernel_26925/1118281666.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we a

In [11]:
train_ordinal = ['LotShape', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
                 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 
                 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence']

train_nominal = ["MSZoning", "Street", "Alley", "LandContour", "LotConfig", "Neighborhood", "Condition1", "Condition2", 
                 "BldgType", "HouseStyle", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType", "Foundation", 
                 "Heating", "CentralAir", "Electrical", "GarageType", "MiscFeature", "SaleType", "SaleCondition"]

In [12]:
ordinal_mapping = {
    "GarageQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
    "Fence": {"GdPrv": 4, "MnPrv": 3, "GdWo": 2, "MnWw": 1, "NA": 0},
    "GarageFinish": {"Fin": 3, "RFn": 2, "Unf": 1, "NA": 0},
    "KitchenQual": {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0},
    "GarageCond": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
    "HeatingQC": {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0},
    "ExterQual": {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0},
    "BsmtCond": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
    "LandSlope": {"Gtl": 2, "Mod": 1, "Sev": 0},
    "ExterCond": {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0},
    "BsmtExposure": {"Gd": 4, "Av": 3, "Mn": 2, "No": 1, "NA": 0},
    "PavedDrive": {"Y": 2, "P": 1, "N": 0},
    "BsmtQual": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
    "LotShape": {"Reg": 3, "IR1": 2, "IR2": 1, "IR3": 0},
    "BsmtFinType2": {"GLQ": 6, "ALQ": 5, "BLQ": 4, "Rec": 3, "LwQ": 2, "Unf": 1, "NA": 0},
    "BsmtFinType1": {"GLQ": 6, "ALQ": 5, "BLQ": 4, "Rec": 3, "LwQ": 2, "Unf": 1, "NA": 0},
    "FireplaceQu": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
    "Utilities": {"AllPub": 3, "NoSewr": 2, "NoSeWa": 1, "ELO": 0},
    "Functional": {"Typ": 7, "Min1": 6, "Min2": 5, "Mod": 4, "Maj1": 3, "Maj2": 2, "Sev": 1, "Sal": 0},
    "PoolQC": {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
}

# Vérifier que les colonnes existent dans data avant transformation
ordinal_columns = [col for col in train_ordinal if col in data.columns]

# Transformer en chaînes de caractères et gérer les valeurs manquantes
for col in ordinal_columns:
    data[col] = data[col].astype(str).str.strip().fillna("NA")

# Transformer les mappings en liste de catégories ordonnées
ordinal_categories = [list(ordinal_mapping[col].keys()) for col in ordinal_columns]

# Encodage Ordinal
ordinal_encoder = OrdinalEncoder(categories=ordinal_categories, 
                                 handle_unknown="use_encoded_value", unknown_value=-1)

data[ordinal_columns] = ordinal_encoder.fit_transform(data[ordinal_columns])


# Encodage One-Hot
#data = pd.get_dummies(data, columns=nominal_columns, dtype=int)

label_encoders = {}
for col in train_nominal:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))
    label_encoders[col] = le

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Sélectionner les features numériques de data
numeric_features = data.select_dtypes(include=['int64', 'float64']).columns

# Pipeline pour la standardisation
pipeline = Pipeline([
    ('scaler', StandardScaler())
])

# Appliquer la standardisation sur les features numériques de data
data[numeric_features] = pipeline.fit_transform(data[numeric_features])

In [14]:
from sklearn.preprocessing import PowerTransformer

# Sélectionner les colonnes numériques
numeric_value = data.select_dtypes(include=['float64', 'int64'])

# Calculer la skewness pour chaque feature numérique
skewness = numeric_value.apply(lambda x: x.skew()).sort_values(ascending=False)

# Définir un seuil de skewness
skew_threshold = 0.75

# Sélectionner les features fortement biaisées
highly_skewed = skewness[skewness > skew_threshold].index
print(f"🔍 Found {len(highly_skewed)} highly skewed features:\n", skewness[highly_skewed])

# Appliquer la transformation Yeo-Johnson
pt = PowerTransformer(method="yeo-johnson")
data[highly_skewed] = pt.fit_transform(data[highly_skewed])

print("✅ Yeo-Johnson transformation applied successfully!")
new_skewness = data[highly_skewed].apply(lambda x: x.skew())
print("📉 Skewness after transformation:\n", new_skewness)


🔍 Found 45 highly skewed features:
 Utilities                      22.026510
MiscVal                        21.917095
PoolArea                       18.631364
LowQualFinSF                   13.748856
LotArea                        13.160801
Condition2                     12.328237
Heating                        12.061794
3SsnPorch                      11.359992
RoofMatl                        8.810638
MiscFeature                     5.056290
LandSlope                       4.967243
Functional                      4.962537
KitchenAbvGr                    4.313446
BsmtFinSF2                      4.162834
EnclosedPorch                   3.956537
BsmtHalfBath                    3.939310
ScreenPorch                     3.928081
area                            3.629765
BsmtCond                        3.603067
GarageCond                      3.407841
GarageQual                      3.309125
Condition1                      3.008828
PavedDrive                      2.989064
Lot_Occupation       

In [15]:


# Diviser les données en train et test
tra = data.iloc[:len(train), :]  # 1460 premières lignes pour l'entraînement
tes = data.iloc[len(train):, :]  # Le reste pour le test

# Extraire SalePrice pour l'entraînement (pour concaténer avec les features d'entraînement)
y_S = y_SalePrice.iloc[:len(train)]  # Pas besoin de `:` supplémentaire ici

# Ajouter y_SalePrice à `tra` (en tant que nouvelle colonne)
tra = pd.concat([tra, y_S], axis=1)

# Vérifier la forme des ensembles train et test
print(tra.shape, tes.shape)

# Sauvegarder les fichiers résultants
tra.to_csv('final_data_nv.csv', index=False)
tes.to_csv('final_test_data_nv.csv', index=False)


(1449, 106) (1459, 105)
